In [5]:
!wget https://github.com/naveenkumar12624/Mini-Project_Age_and_Gender_Prediction/raw/main/opencv_face_detector.pbtxt
!wget https://github.com/naveenkumar12624/Mini-Project_Age_and_Gender_Prediction/raw/main/opencv_face_detector_uint8.pb
!wget https://github.com/naveenkumar12624/Mini-Project_Age_and_Gender_Prediction/raw/main/age_deploy.prototxt
!wget https://github.com/naveenkumar12624/Mini-Project_Age_and_Gender_Prediction/raw/main/age_net.caffemodel
!wget https://github.com/naveenkumar12624/Mini-Project_Age_and_Gender_Prediction/raw/main/gender_deploy.prototxt
!wget https://github.com/naveenkumar12624/Mini-Project_Age_and_Gender_Prediction/raw/main/gender_net.caffemodel
    
    
import requests

urls = [
    "https://github.com/naveenkumar12624/Mini-Project_Age_and_Gender_Prediction/raw/main/opencv_face_detector.pbtxt",
    "https://github.com/naveenkumar12624/Mini-Project_Age_and_Gender_Prediction/raw/main/opencv_face_detector_uint8.pb",
    "https://github.com/naveenkumar12624/Mini-Project_Age_and_Gender_Prediction/raw/main/age_deploy.prototxt",
    "https://github.com/naveenkumar12624/Mini-Project_Age_and_Gender_Prediction/raw/main/age_net.caffemodel",
    "https://github.com/naveenkumar12624/Mini-Project_Age_and_Gender_Prediction/raw/main/gender_deploy.prototxt",
    "https://github.com/naveenkumar12624/Mini-Project_Age_and_Gender_Prediction/raw/main/gender_net.caffemodel"
]

for url in urls:
    response = requests.get(url)
    filename = url.split("/")[-1]
    with open(filename, 'wb') as f:
        f.write(response.content)
    print(f"Downloaded {filename}")


'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.


Downloaded opencv_face_detector.pbtxt
Downloaded opencv_face_detector_uint8.pb
Downloaded age_deploy.prototxt
Downloaded age_net.caffemodel
Downloaded gender_deploy.prototxt
Downloaded gender_net.caffemodel


In [ ]:
import cv2
import os
import math
import argparse
import sys

def highlightFace(net, frame, conf_threshold=0.7):
    frameOpencvDnn = frame.copy()
    frameHeight = frameOpencvDnn.shape[0]
    frameWidth = frameOpencvDnn.shape[1]
    blob = cv2.dnn.blobFromImage(frameOpencvDnn, 1.0, (300, 300), [104, 117, 123], True, False)

    net.setInput(blob)
    detections = net.forward()
    faceBoxes = []
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > conf_threshold:
            x1 = int(detections[0, 0, i, 3] * frameWidth)
            y1 = int(detections[0, 0, i, 4] * frameHeight)
            x2 = int(detections[0, 0, i, 5] * frameWidth)
            y2 = int(detections[0, 0, i, 6] * frameHeight)
            faceBoxes.append([x1, y1, x2, y2])
            cv2.rectangle(frameOpencvDnn, (x1, y1), (x2, y2), (0, 255, 0), int(round(frameHeight / 150)), 8)
    return frameOpencvDnn, faceBoxes


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument('--image', help="Path to image file. If not specified, webcam will be used", default=None)

    # Check if we are running in Jupyter or command line
    if sys.argv[0].endswith('ipykernel_launcher.py'):
        args = parser.parse_args(args=[])  # Bypass unrecognized arguments in Jupyter
    else:
        args = parser.parse_args()
        

    faceProto = "opencv_face_detector.pbtxt"
    faceModel = "opencv_face_detector_uint8.pb"
    ageProto = "age_deploy.prototxt"
    ageModel = "age_net.caffemodel"
    genderProto = "gender_deploy.prototxt"
    genderModel = "gender_net.caffemodel"

    print("Checking if files exist:")
    print("Face Model Path Exists:", os.path.isfile(faceModel))
    print("Face Proto Path Exists:", os.path.isfile(faceProto))
    print("Age Model Path Exists:", os.path.isfile(ageModel))
    print("Age Proto Path Exists:", os.path.isfile(ageProto))
    print("Gender Model Path Exists:", os.path.isfile(genderModel))
    print("Gender Proto Path Exists:", os.path.isfile(genderProto))

    MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
    ageList = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
    genderList = ['Male', 'Female']

    # Load the models for face, age, and gender detection
    faceNet = cv2.dnn.readNet(faceModel, faceProto)
    ageNet = cv2.dnn.readNet(ageModel, ageProto)
    genderNet = cv2.dnn.readNet(genderModel, genderProto)

    # Capture video from file or webcam
    video = cv2.VideoCapture(args.image if args.image else 0)
    padding = 20

    while cv2.waitKey(1) < 0:
        hasFrame, frame = video.read()
        if not hasFrame:
            cv2.waitKey()
            break

        # Detect faces
        resultImg, faceBoxes = highlightFace(faceNet, frame)
        if not faceBoxes:
            print("No face detected")

        # Loop over detected faces
        for faceBox in faceBoxes:
            face = frame[max(0, faceBox[1] - padding): min(faceBox[3] + padding, frame.shape[0] - 1),
                         max(0, faceBox[0] - padding): min(faceBox[2] + padding, frame.shape[1] - 1)]

            blob = cv2.dnn.blobFromImage(face, 1.0, (227, 227), MODEL_MEAN_VALUES, swapRB=False)

            # Predict gender
            genderNet.setInput(blob)
            genderPreds = genderNet.forward()
            gender = genderList[genderPreds[0].argmax()]
            print(f'Gender: {gender}')

            # Predict age
            ageNet.setInput(blob)
            agePreds = ageNet.forward()
            age = ageList[agePreds[0].argmax()]
            print(f'Age: {age[1:-1]} years')

            # Display results on image
            cv2.putText(resultImg, f'{gender}, {age}', (faceBox[0], faceBox[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2, cv2.LINE_AA)
            cv2.imshow("Detecting age and gender", resultImg)


Checking if files exist:
Face Model Path Exists: True
Face Proto Path Exists: True
Age Model Path Exists: True
Age Proto Path Exists: True
Gender Model Path Exists: True
Gender Proto Path Exists: True
No face detected
No face detected
No face detected
No face detected
No face detected
No face detected
No face detected
No face detected
No face detected
No face detected
No face detected
No face detected
No face detected
No face detected
No face detected
No face detected
No face detected
No face detected
No face detected
No face detected
No face detected
No face detected
No face detected
No face detected
No face detected
No face detected
No face detected
No face detected
No face detected
No face detected
No face detected
No face detected
No face detected
No face detected
No face detected
No face detected
No face detected
No face detected
No face detected
No face detected
No face detected
No face detected
No face detected
No face detected
Gender: Female
Age: 25-32 years
No face detected
Ge